In [5]:
import pandas as pd

In [156]:
import ast
df = pd.read_csv('locations.csv')
df

,Place,passenger side,driver side,back
0,Front bumper,"(266,168)",NaN,NaN
1,Rear bumper,"(45,158)","(305,70)",NaN
2,Front fender,"(233,145)",NaN,NaN
3,Rear fender,"(80,140)",NaN,NaN
4,Hood,"(168,162)",NaN,NaN
5,Trunk,"(37,118)",NaN,NaN
6,Roof,"(133,82)",NaN,NaN
7,door,"(198,217)",NaN,NaN
8,Side window,NaN,NaN,NaN
9,NaN,"(210,101)",NaN,NaN


In [99]:
from difflib import SequenceMatcher

# List of keywords to search for
keywords = ['Crack', 'Dent', 'Hole', 'Scratch']

# List of places to check for
places = ['windshield', 'rear bumper', 'door', 'window', 'fender', 'hood', 'roof', 'trunk', 'mirror','Side windows']

# Input sentence
input_sentence = "There is a crack on the windshield and a dent on the rear bumper."

# Function to calculate similarity ratio between two strings
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

# Function to extract damage indications and their corresponding places
def extract_damage_indications(sentence, keywords, places):
    # Convert the sentence to lowercase for case-insensitive search
    sentence = sentence.lower()
    # Extract damage indications and their corresponding places
    damage_indications = []
    for place in places:
        for keyword in keywords:
            if place.lower() in sentence and keyword.lower() in sentence:
                damage_indication = {place: keyword}
                # Check if the damage indication for the same place already exists in the list
                # If yes, update the keyword with higher similarity score
                existing_damage_indications = [di for di in damage_indications if place in di.keys()]
                if existing_damage_indications:
                    existing_damage_indication = existing_damage_indications[0]
                    existing_keyword = existing_damage_indication[place]
                    similarity_existing = similar(existing_keyword, keyword)
                    similarity_current = similar(existing_keyword, keyword)
                    if similarity_current > similarity_existing:
                        existing_damage_indication[place] = keyword
                else:
                    damage_indications.append(damage_indication)
    return damage_indications

# Call the function and print the extracted damage indications and their corresponding places
damage_indications = extract_damage_indications(input_sentence, keywords, places)
# print("Extracted Damage Indications:")
# print(damage_indications)


In [138]:
# List of side parameters to search for
from collections import OrderedDict
sent_pos = OrderedDict()
sides = ['driver side', 'passenger side', 'back']
input_sentence = "driver side There is a dent on the rear bumper. passenger side hole in the hood, crack on door."
max_s = len(input_sentence)
cnt = 0
sentence=input_sentence
for my_side in sides :
    position = sentence.lower().find(my_side.lower())
    if position != -1:
        sent_pos[my_side]=(position,max_s)
        if cnt:
            sent_pos[list(sent_pos.keys())[cnt-1]]=(sent_pos[list(sent_pos.keys())[cnt-1]][0],position-1)
        cnt+=1
my_res = {}
for i in sent_pos:
    sentence = input_sentence[sent_pos[i][0]:sent_pos[i][1]]
#     print (sentence)
    damage_indications = extract_damage_indications(sentence, keywords, places)
#     print(damage_indications)
    my_res[i]=damage_indications
print(my_res)    

{'driver side': [{'rear bumper': 'Dent'}], 'passenger side': [{'door': 'Crack'}, {'hood': 'Crack'}]}


In [157]:
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw 
image = Image.open("cars.png")

# Create draw object
width, height = image.size
draw = ImageDraw.Draw(image)
format = image.format
font = font = ImageFont.truetype("Arial Unicode.ttf", 24)

# Define text color and position
color = (255, 0, 0)  # white

# Draw text on image
for dm_l in my_res:
    for dm_l2 in my_res[dm_l]:
#         draw.text(position, text, color, font=font)
        print(dm_l2)
        place = df[df['Place'].str.lower()==list(dm_l2.keys())[0].lower()][dm_l]
        position = eval(list(place)[0])
        text = list(dm_l2.values())[0][0]
        draw.text(position, text, color, font=font)



# draw.rectangle([(50, 50), (150, 150)], outline="red", width=5)

# Save the edited image with the same dimensions and format as the original image
image.save("edited.png", format=format, size=(width, height))
# Save the edited image
# image.save("edited.png")


{'rear bumper': 'Dent'}
{'door': 'Crack'}
{'hood': 'Crack'}
